# Simple Procedural Content Generation for Space Engineers

Create functional spaceships in Space Engineers by PCG using the [SE API](https://github.com/iv4xr-project/iv4xr-se-plugin).

Additional information for SE can be found [here](https://github.com/iv4xr-project/iv4xr-se-plugin/tree/main/JvmClient#basic-space-engineers-engine-info).

SE API calls are listed [here](https://github.com/iv4xr-project/iv4xr-se-plugin/blob/main/Source/Ivxr.SePlugin/Communication/AustinJsonRpcSpaceEngineers.cs).

**NOTE**: Have the game *running*; leaving the game simply paused breaks the API (no calls are accepted). *Suggestion*: Use windowed mode, press `Start` after the scenario is loaded and return to this notebook.

Import required packages

In [113]:
import ast
import json
import numpy as np
import random
import os
import socket
import time

from enum import Enum
from typing import List, Optional, Dict, Any, Tuple

## API Interfacing

Variables for the API:

In [114]:
host = 'localhost'
port = 3333

API common structs/classes

In [115]:
def vec2(x: int,
         y: int) -> Dict[str, int]:
    return {
        "X": x,
        "Y": y
    }

def vec3(x: int,
         y: int,
         z: int) -> Dict[str, int]:
    return {
        "X": x,
        "Y": y,
        "Z": z
    }

def vec2f(x: float,
         y: float) -> Dict[str, float]:
    return {
        "X": x,
        "Y": y
    }

def vec3f(x: float,
         y: float,
         z: float) -> Dict[str, float]:
    return {
        "X": x,
        "Y": y,
        "Z": z
    }

def sum_vecs(a: Dict[str, Any],
             b: Dict[str, Any]) -> Dict[str, Any]:
    return {
        "X": a["X"] + b["X"],
        "Y": a["Y"] + b["Y"],
        "Z": a["Z"] + b["Z"],
    }

class Orientation(Enum):
    UP = vec3f(0., 1., 0.)
    DOWN = vec3f(0., -1., 0.)
    RIGHT = vec3f(1., 0., 0.)
    LEFT = vec3f(-1., 0., 0.)
    FORWARD = vec3f(0., 0., -1.)
    BACKWARD = vec3f(0., 0., 1.)

Character position to camera position offset

In [116]:
character_camera_dist = vec3f(0., 1.6369286, 0.)

### Block class

In [117]:
class Block:
    def __init__(self,
                 block_type: str,
                 orientation_forward: Orientation = Orientation.FORWARD,
                 orientation_up: Orientation = Orientation.UP):
        self.block_type = block_type
        self.orientation_forward = orientation_forward
        self.orientation_up = orientation_up
        self.position = vec3f(0., 0., 0.)
    
    def __str__(self) -> str:
        return f'{self.block_type} at {self.position}; OF {self.orientation_forward}; OU {self.orientation_up}'
    
    def __repr__(self) -> str:
        return f'{self.block_type} at {self.position}; OF {self.orientation_forward}; OU {self.orientation_up}'

blocks_dims = {
    's': 0.5,  # small
    'n': 1.,  # normal
    'l': 2.5  # large
}

def get_block_size(block: Block) -> str:
    return 's' if block.block_type.startswith('Small') else ('l'if block.block_type.startswith('Large') else 'n')

def get_block_dim(block: Block) -> float:
    return blocks_dims[get_block_size(block)]

### Structure class

A structure is a 3D matrix filled with `Block`s. Currently, the `Structure` is a fixed-size 3D matrix of smallest size cubes (`0.5`); larger cubes are inserted once and occupy the neighbouring spaces.

In [118]:
class Structure:
    def __init__(self,
                 origin: Dict[str, Any],
                 orientation_forward: Dict[str, Any],
                 orientation_up: Dict[str, Any],
                 dimensions: Tuple[int, int, int] = (10, 10, 10)) -> None:
        self._VALUE = 0.5
        self.origin_coords = origin
        self.orientation_forward = orientation_forward
        self.orientation_up = orientation_up
        self._structure: np.ndarray = 0.5 * np.ones(dimensions)  # keeps track of occupied spaces
        self._blocks: Dict[float, Block] = {}  # keeps track of blocks info
            
    def add_block(self,
                  block: Block,
                  grid_position: Tuple[int, int, int]) -> None:
        k, j, i = grid_position
        assert self._structure[i][j][k] == 0.5, f'Error when attempting to place block {block.block_type}: space already occupied.'
        block_size = get_block_size(block)
        if block_size != 's':
            # update neighbouring cells
            r = 5 if block_size == 'l' else 2
            n, target = np.sum(self._structure[i:i+r, j:j+r, k:k+r]) - self._VALUE, self._VALUE * ((r ** 3) - 1)
            assert n == target, f'Error when placing block {block.block_type}: block would intersect already existing block(s).'
            self._structure[i:i+r, j:j+r, k:k+r] = np.zeros((r,r,r))
        block_id = float(self._VALUE + len(self._blocks.keys()) + 1)  # unique sequential block id as key in dict
        self._structure[i, j, k] = block_id
        self._blocks[block_id] = block
        # update block position
        dx, dy, dz = 0., 0., 0.
        if i > 0:
            for e in self._structure[0:i, j, k]:
                dz += get_block_dim(self._blocks[e]) if e > self._VALUE else e
        if j > 0:
            for e in self._structure[i, 0:j, k]:
                dy += get_block_dim(self._blocks[e]) if e > self._VALUE else e
        if k > 0:
            for e in self._structure[i, j, 0:k]:
                dx += get_block_dim(self._blocks[e]) if e > self._VALUE else e        
        block.position = sum_vecs(self.origin_coords, vec3f(dx, dy, dz))
    
    def get_all_blocks(self) -> List[Block]:
        return self._blocks.values()

### API interfacing methods

In [119]:
def generate_json(method: str,
                  params: Optional[List[Any]] = None) -> Dict[str, Any]:
    return {
        "jsonrpc": "2.0",
        "method": method,
        "params": params if params else [],
        "id": random.getrandbits(16)
    }

def compactify_jsons(jsons: List[Dict[str, Any]]) -> List[str]:
    """ JSONs should be single line & compact formatted before calling the API. (@Karel Hovorka)"""
    compacted_jsons = ''
    for j in jsons:
        compacted_jsons += json.dumps(obj=j,
                               separators=(',', ':'),
                               indent=None)
        compacted_jsons += '\r\n'

    return compacted_jsons

# Modified from https://www.binarytides.com/receive-full-data-with-the-recv-socket-function-in-python/
def recv_with_timeout(s: socket.socket,
                      timeout: int = 2):
    #make socket non blocking
    s.setblocking(0)
    #total data partwise in an array
    total_data = [];
    #beginning time
    begin = time.time()
    while True:
        # timeout termination
        if total_data and time.time() - begin > timeout:
            break
        # wait to get data
        elif time.time() - begin > timeout * 2:
            break
        # recv
        try:
            data = s.recv(8192).decode("utf-8")
            if data:
                # early break if socket returns the same data
                if len(total_data) > 1 and data == total_data[-1]:
                    break
                # append new data
                total_data.append(data)
                # change the beginning time for measurement
                begin = time.time()
            else:
                #sleep to indicate a gap
                time.sleep(0.1)
        except:
            # note: Exceptions are thrown due to socket not reading anything before and after data is passed
            pass
    # join all parts to make final string
    return ''.join(total_data)

def call_api(jsons: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    # open socket for communication
    s = socket.socket(family=socket.AF_INET,
                      type=socket.SOCK_STREAM)
    s.connect((host, port))    
    # send methods as compacted JSON bytearray
    s.sendall(compactify_jsons(jsons=jsons).encode('utf-8'))
    # get response
    res = recv_with_timeout(s)
    # close socket
    s.close()
    # due to TCP streming packets, it's possible some JSON-RPC responses are the same;
    # workaround: identify unique JSON-RPC responses by unique id
    return [json.loads(x) for x in list(set(res.strip().split('\r\n')))]

## Game speed

We can toggle the game speed by togglin the game's frame limit.

**NOTE**: Switch between `GameMode.PLACING` (fast) and `GameMode.EVALUATING` (slow).

In [120]:
class GameMode(Enum):
    PLACING = False
    EVALUATING = True

def toggle_gamemode(mode: GameMode) -> None:
    # "we plan to change the API and rename this function in near future" (@Karel Hovorka)
    call_api(jsons=[generate_json(method="Admin.SetFrameLimitEnabled",
                                  params=[mode.value])])

In [121]:
toggle_gamemode(mode=GameMode.EVALUATING)

## Structure placement

[Blocks DefintionIDs](https://github.com/iv4xr-project/iv4xr-se-plugin/tree/main/JvmClient#definitionid-id) are used to specify which block to place. Here we either ask the API directly to provide us the list of available DefintionIDs or we can load the .JSON which contains them (faster in future runs).

**NOTE**: This cell produces the global variable `block_definitions`, a map between the block type and the API-supported ID to use in the request `params` field.

In [122]:
if not os.path.exists('./block_definitions.json'):
    # poll API for block definition ids
    jsons = [ 
        generate_json(method="Definitions.BlockDefinitions")
        ]
    res = json.loads(call_api(jsons=jsons))
    # transform to map of type:id
    block_definitions = {v['Type']:v for v in [entry['DefinitionId'] for entry in res['result']]}
    with open('./block_definitions.json', 'w') as f:
        json.dump(block_definitions, f)
else:
    with open('./block_definitions.json', 'r') as f:
        block_definitions = json.load(f)

Structure creation and placement.

In [123]:
def get_base_values():
    obs = call_api(jsons=[generate_json(method="Observer.Observe")])[0]
    base_position = obs['result']['Position']
    orientation_forward = obs['result']['OrientationForward']
    orientation_up = obs['result']['OrientationUp']
    return base_position, orientation_forward, orientation_up

def place_blocks(blocks: List[Block]) -> None:
    # prepare jsons
    jsons = [generate_json(method="Admin.Blocks.PlaceAt",
                           params={
                               "blockDefinitionId": block_definitions[block.block_type],
                               "position": block.position,
                               "orientationForward": block.orientation_forward.value,
                               "orientationUp": block.orientation_up.value
                           }) for block in blocks]
    # place blocks
    call_api(jsons=jsons)

Example of structure placement.

At the moment, the structure spawns but is not functional as the blocks are not part of the same game grid. Waiting to hear back from GoodAI.

In [124]:
base_position, orientation_forward, orientation_up = get_base_values()
structure = Structure(origin=base_position,
                      orientation_forward=orientation_forward,
                      orientation_up=orientation_up,
                      dimensions=(10, 10, 10))
structure.add_block(block=Block(block_type='LargeBlockSmallGenerator',
                                orientation_forward=Orientation.LEFT,
                                orientation_up=Orientation.DOWN),
                    grid_position=(0, 0, 0)
                    )
structure.add_block(block=Block(block_type='LargeBlockCockpitSeat',
                                orientation_forward=Orientation.UP,
                                orientation_up=Orientation.RIGHT
                                ),
                                grid_position=(0, 5, 0))

place_blocks(structure.get_all_blocks())